<a href="https://colab.research.google.com/github/RegaipKURT/QuantumProgramming/blob/master/Q_SVM.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Q-SVM (Quantum Support Vector Machine Algoritması)

Quantum bilgisayarları ve quantum programlama çok yeni bir alan olmasıyla beraber, hem merak uyandırmakta hem de neler yapılabileceğinin anlaşılması için deneysel çalışmalara sahne olmaktadır. Bu deneysel çalışma alanlarından birisi de makine öğrenmesi ve derin öğrenme algoritmalarının, hatta yapay zekanın kuantum bilgisayarlar üzerinde denenmesi ile ilgili yapılan çalışmalardır. Ancak derin öğrenme yöntemlerinin ve yapay zeka ile ilgili çalışmaların bile henüz başındayken bunların kuantum bilgisayarlarda uygulanması, şimdilik pratik kullanımlardan ziyade akademik ve teorik çalışmalar olarak devam etmektedir. 


Araştırdığım kadarıyla QSVM algoritmasını uygulamak 2 farklı yöntemle gerçekleştirelebilir. Bu yöntemlerden 1.'si sadece kernel fonksiyonunu quantum bilgisayarında çalıştırmak verileri ise klasik bir bilgisayardan ve bilindik özellik uzayından konvensiyonel yöntemlerle indirgeyerek almak. İkincisi ise hem özellik uzayında verilerin indirgenmesini ve verilerin etiketlerini hem de kernel fonksiyonunu quantum bilgisayarları üzerinde yapmak.

Bu iki yöntem arasındak farkı anlamak için ilk önce SVM'in klasik halinin verileri nasıl sınıflandırdığını bilmemiz gerekiyor. Bunu bildiğimizi varsayarak özellik uzayında verilerin boyut artırılarak  nasıl birbirinden ayrıldığını gösteren bir görselle hatırlayalım: 

![alt text](https://www.researchgate.net/profile/Katarzyna_Matek/publication/235892711/figure/fig1/AS:299920994127876@1448518145475/Illustration-of-the-operation-of-the-SVM-algorithm-The-input-data-on-the-left-side.png)

Yukarıda bahsettiğimiz iki yöntemin farkı birincisinde özellik uzayımızın da kuantum devrelerinde hesaplanmasıdır. Kuantum özellik uzayı da aşağıdaki şekilde  gösterilebilir.


![alt text](https://raw.githubusercontent.com/RegaipKURT/QuantumProgramming/master/qc.png)


Her iki yöntemde de ortak nokta kernel fonksiyonunun geleneksel yolla değil quantum hesaplamaları ile elde edilmesidir. Fakat başarı oranını artırmak için verilerin etiketlerinin süperpozisyon durumunda sağlanabilmeleri gerekiyor. Bunu yapabilmek için ise verilerin de bir quantum devresinden okunması gerekmekle beraber, biz burada bu veri sağlama yöntemini uygulayacak imkana şu an sahip değiliz. Ama deneysel çalışmalarda elde edilen sonuçlar bu yöntemle %100 tahmin başarısı sağlanabileceğini gösteriyor. (Tabi %100 başarı elde edilebilecek düzenli bir veri kullanılarak bu yapılmış.) 

Her ne kadar verileri quantum bilgisayarından sağlamak imkanımız olmasa bile, qiskit kütüphanesi içine kuantum özellik uzayını kullanarak verilerin birbirinden ayrılmasını sağlayacak bir fonksiyon yerleştirilmiş. Biz de feature_map isimli bu kütüphaneyi aşağıda kullanacağız. 

Burada bahsettiklerimle ilgili çalışmaya ait makalenin linkinden akademik çalışmayı inceleyebilir ve makalenin alıntılarından başka kaynaklara da ulaşabilirsiniz. Konuyla ilgili başka çalışmalar da yapılmış. Aşağıda linkini verdiğim makaleye qiskit.aqua içinden de referans verilmiş.

---

Quantum-SVM algoritmasının makalesi:  https://arxiv.org/pdf/1804.11326.pdf

# Niye Q-SVM'e gerek duymuşlar? Neden böyle çalışmalar yapılıyor?

Klasik makine öğrenmesi algoritmaları dururken neden kuantum bilgisayarları ve algoritmaları kullanılıyor peki? Bu sorunun iki cevabı var: 1.'si kuantum dolaşıklığı gibi kauntum fiziği özelliklerinden faydalanmak, 2.'si ise kuantum bilgisayarlarının hesaplama gücünü kullanmak. 
 
SVM algoritmasında verilerin birbirine yakın noktalarından destek noktaları oluşturarak verileri sınıflandırma yoluna gidiliyor. Yukarıda özellik uzaylarının fotoğrafları bulunuyor ve anlaşılabilir  ki bazen iki veya üç boyut kullanarak verileri sınıflandırmak mümkün olamayabilir. Bunun için de n-boyutlu bir uzaya doğru boyut artıma işlemi yapmak ve verilerimizi bu uzayda temsil eden noktalarla birbirinden ayırmak gerekebiliyor. Fakat çok boyutlu uzaylarda çalıştıkça klasik bilgisayarlarda hesaplama gücümüz de yetersiz gelmeye başlıyor. 

#### Eğer qiskit kütüphanesi yüklü değilse aşağıdaki komut ile yüklememiz gerekmektedir.

In [0]:
!pip install qiskit

## Kütüphanelerin import edilmesi

İlk olarak qiskit.aqua içerisinden qsvm algoritmasını kullanacağız. Fakat başka kütüphanelerde bunun alternatifi olan qsvm algoritmaları var. Kullanacağımız veriseti ise makine öğrenmesi çalışmalarında en sık kullanılan verisetlerinden biri olan iris veriseti.

In [0]:
import numpy as np
from sklearn.datasets import load_iris
from sklearn.model_selection import train_test_split
import qiskit
from qiskit import BasicAer, Aer
from qiskit.aqua import QuantumInstance
from qiskit.aqua.algorithms import QSVM
from qiskit.aqua.components.multiclass_extensions import one_against_rest, all_pairs
from qiskit.aqua.components.feature_maps import SecondOrderExpansion
from qiskit.aqua.input import ClassificationInput
from qiskit.aqua import run_algorithm

backend = Aer.get_backend('qasm_simulator')
iris = load_iris()

Algoritmayı çalıştırmak çok uzun sürdüğü için ibm'in bilgisayarları üzerinde çalıştırıp uzun süre beklememek ve orayı da meşgul etmemek için qasm_simulator üzerinde çalıştırıyorum. 

## Verisetinin Yüklenmesi

Kullancağımız veriseti olan iris hedef değişken olarak 3 ayrı sınıf ve 4 özellik kolonundan oluşuyor. 

In [11]:
X, Y = iris.data, iris.target
print(X.shape)
print(len(set(Y)))
train_x, test_x, train_y, test_y = train_test_split(X, Y, test_size=0.2)
num_features = 4
training_size = 120
test_size = 30
feature_map = SecondOrderExpansion(feature_dimension=num_features, depth=3)
# feature_map ile yukarıda bahsettiğimiz kuantum özellik uzayını kullanacağımız fonksiyonu tanımladık.

(150, 4)
3


Yukarıda feature map tanımladık ama aşağıda zaten parametre olarak vereceğimizden dolayı burada tanımlamamıza gerek yoktu. Örnek olması açısından şimdilik her ikisi de kalsın.

In [0]:
params = {
            'problem': {'name': 'classification', 'random_seed': 794},
            'algorithm': {
                'name': 'QSVM',
            },
            'backend': {'shots': 1},
            'multiclass_extension': {'name': 'OneAgainstRest'},
            'feature_map': {'name': 'SecondOrderExpansion', 'depth': 3, 'entangler_map': [[1, 0]]}
            #'feature_map': {'name': 'SecondOrderExpansion', 'depth': 3}
        }
training_dataset={'A':train_x[train_y==0],
                'B':train_x[train_y==1],
                'C':train_x[train_y==2]}

test_dataset={'A':test_x[test_y==0],
                        'B':test_x[test_y==1],
                        'C':test_x[test_y==2]}

total_arr = np.concatenate((test_dataset['A'],test_dataset['B'],test_dataset['C']))
alg_input = ClassificationInput(training_dataset, test_dataset, total_arr)

In [0]:
result = run_algorithm(params, algo_input=alg_input, backend=backend)

### Sonuçları yazdırıp nasıl bir çıktı aldığımızı görelim 

In [7]:
result

{'predicted_classes': ['A',
  'A',
  'A',
  'A',
  'A',
  'A',
  'A',
  'A',
  'A',
  'A',
  'A',
  'A',
  'B',
  'B',
  'B',
  'B',
  'B',
  'C',
  'C',
  'B',
  'A',
  'B',
  'C',
  'B',
  'B',
  'B',
  'B',
  'C',
  'A',
  'C'],
 'predicted_labels': array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 2, 2, 1, 0, 1,
        2, 1, 1, 1, 1, 2, 0, 2]),
 'test_success_ratio': 0.7,
 'testing_accuracy': 0.7}

In [9]:
print("Tahmin edilen Sınıflar:   ", result["predicted_labels"])
print("Sınıfların gerçek değeri: ", test_y)

Tahmin edilen Sınıflar:    [0 0 0 0 0 0 0 0 0 0 0 0 1 1 1 1 1 2 2 1 0 1 2 1 1 1 1 2 0 2]
Sınıfların gerçek değeri:  [0 0 1 1 2 2 2 0 2 2 1 0 0 0 2 2 1 2 0 0 1 0 1 0 0 1 1 1 0 0]


Bizim elde ettiğimiz sonuçlar pek iç acıcı değil. Bu sonuçlarda teknik olarak tam anlamıyla bir QSVM çalıştırmamış olmamızın da etkisi var. Ama yukarıda verdiğimiz makaleyi inceleyerek daha kesin sonuçlar elde etmenin mümkün olduğunu görebilirsiniz. 

İlerleyen 10 yılda teknolojinin gelişmesi ve donanımsal engellerin de birer birer ortadan kalkmasıyla beraber kuantum bilgisayarlarında daha gelişmiş algoritmaları daha kesin sonuçlar alacak şekilde çalıştırmamız mümkün olacaktır. 